Physical Emission Intensity Data Analysis

Setup: Paths and Logging

In [1]:
# core imports
import os
import sys
import logging
import pandas as pd
import numpy as np

notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

# set up logging to print to notebook output
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(levelname)s: %(message)s")
logger = logging.getLogger()

Imports:

In [2]:
import src.config as config
from src.data_loader import DataLoader
from src.coordinate_converter import CoordinateConverter
from src.spatial_matcher     import SpatialMatcher
from src.analyzers           import (
    PowerPlantAnalyzer,
    CapacityFactorAnalyzer,
    EmissionAnalyzer,
    EnergyCalculator,
    EmissionCalculator,
)


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Data Loading

In [3]:
#Global datasets
gpp = DataLoader.load_global_power_plants()
emu = DataLoader.load_ember_power_plants()
electricity_generation_df = DataLoader.load_excel(config.POWERCO_XLSX, sheet_name=config.ELECTRICITY_GENERATION_XLSX_SHEET)
capacity_factor_df = DataLoader.load_excel(config.CAPACITY_FACTOR_XLSX, sheet_name=config.CAPACITY_FACTOR_XLSX_SHEET)
offshore_wind_df = DataLoader.load_excel(config.OFFSHORE_WIND_XLSX, sheet_name=config.OFFSHORE_WIND_XLSX_SHEET)
#Client data
powerco_df = DataLoader.load_excel(config.POWERCO_XLSX, sheet_name=config.POWERCO_XLSX_SHEET)

Geographic matching to join power plants' datasets

In [4]:
gpp_matched = SpatialMatcher.match_locations(gpp, emu, tol=0.02)
logger.info("Matched %d plants", len(gpp_matched))

2025-05-15 17:35:31,124 INFO: Matched 34 of 180 points under 0.020°
2025-05-15 17:35:31,140 INFO: Matched 34 plants


Calculation of average emissions per MW for Gas Plants

In [5]:
#Convert gpp_match tons of emissions of CO2 to kg of CO2
gpp_matched["Emissions (tCO2e)"] = gpp_matched["Emissions (tCO2e)"] * config.TONS_TO_KG
gpp_matched.rename(columns={"Emissions (tCO2e)": "Emissions (kgCO2e)"}, inplace=True)
emissions = EmissionAnalyzer.calculate_emissions_per_mw(gpp_matched)
gas_plants_avg_kg_emission_per_mw = EmissionAnalyzer.calculate_average_emissions_per_mw(emissions)
logger.info("Avg kg CO2e per MW: %.2f", gas_plants_avg_kg_emission_per_mw)


2025-05-15 17:35:31,178 INFO: Avg kg CO2e per MW: 7168687.09


Capacity Factors for Gas Power Plants in Germany

In [6]:
germany_gas_power_plants_df = gpp[['Name', 'Capacity (MW)', 'Generation (GWh)']].copy()
germany_gas_power_plants_df ["Generation (GWh)"] = germany_gas_power_plants_df["Generation (GWh)"]* config.GWH_TO_MWH
germany_gas_power_plants_df.rename(columns={'Generation (GWh)': 'generation(MWh)'}, inplace=True)
germany_gas_power_plants_df.rename(columns={'Capacity (MW)': 'capacity(MW)'}, inplace=True)
germany_gas_power_plants_df = CapacityFactorAnalyzer.calculate_capacity_factor(germany_gas_power_plants_df, 'generation(MWh)', 'capacity(MW)')
average_german_gas_capacity_factor = CapacityFactorAnalyzer.average_capacity_factor(germany_gas_power_plants_df, 'Capacity Factor')
logger.info("Avg capacity factor for gas power plants in Germany: %.2f", average_german_gas_capacity_factor)

2025-05-15 17:35:31,209 INFO: Avg capacity factor for gas power plants in Germany: 0.44


Capacity Factors for Solar Power Plants in Germany

In [7]:
# Filter and calculate capacity factors for solar parks in Germany
solar_parks_germany_df = capacity_factor_df[(capacity_factor_df['Country'] == 'Germany') & (capacity_factor_df['Energy Source'] == 'Solar Park')].copy()
avg_solar_capacity_factor = CapacityFactorAnalyzer.average_capacity_factor(solar_parks_germany_df, 'Value')
logger.info("Avg capacity factor for solar parks in Germany: %.2f", avg_solar_capacity_factor)

2025-05-15 17:35:31,260 INFO: Avg capacity factor for solar parks in Germany: 0.12


Capacity Factors for Offshore Wind Parks in the North Sea

In [8]:
offshore_wind_df = CoordinateConverter.convert_coordinates(offshore_wind_df, 'Latitude', 'Longitude')
analyzer = PowerPlantAnalyzer(offshore_wind_df)
north_sea_power_plants = analyzer.filter_north_sea('Latitude', 'Longitude')
avg_offshore_north_sea_capacity_factor = CapacityFactorAnalyzer.average_capacity_factor(north_sea_power_plants, 'Capacity factor')
logger.info("Avg capacity factor for offshore wind in North Sea: %.2f", avg_offshore_north_sea_capacity_factor)

2025-05-15 17:35:31,309 INFO: Avg capacity factor for offshore wind in North Sea: 0.44


Calculate load proportions: The proportion of energy generated by base load and peak load operations power plants

In [9]:
peak_load_proportion, base_load_proportion = PowerPlantAnalyzer.calculate_load_proportions(electricity_generation_df)
logger.info("Peak load proportion: %.2f", peak_load_proportion)
logger.info("Base load proportion: %.2f", base_load_proportion)
average_capacity_factor_continous_load = (average_german_gas_capacity_factor - (peak_load_proportion * config.PEAK_CAPACITY_FACTOR)) / base_load_proportion

2025-05-15 17:35:31,338 INFO: Peak load proportion: 0.08
2025-05-15 17:35:31,338 INFO: Base load proportion: 0.92


Calculation of Physical Emission Intensity for PowerCo

Bavaria Gas Power Plant base load 100 MW

In [10]:
bavaria_gas_capacity = PowerPlantAnalyzer.select_client_plant_value(powerco_df, 'Power Plant type', 'Gas (Base load)', 'Power Capacity (MW)')
bavaria_gas_energy = EnergyCalculator.calculate_energy_generation(bavaria_gas_capacity, average_capacity_factor_continous_load) #bavaria_gas_capacity * HOURS_IN_YEAR * avg_offshore_capacity_factor
bavaria_gas_emissions = EmissionCalculator.calculate_emission(gas_plants_avg_kg_emission_per_mw, bavaria_gas_capacity)
bavaria_emission_intensity = EmissionCalculator.calculate_emission_intensity(bavaria_gas_emissions, bavaria_gas_energy)
logger.info("Bavaria gas power plant energy generation: %.2f MWh", bavaria_gas_energy)
logger.info("Bavaria gas power plant emissions: %.2f kg CO2e", bavaria_gas_emissions)
logger.info("Bavaria gas power plant emission intensity: %.2f kg CO2e/MWh", bavaria_emission_intensity)

2025-05-15 17:35:31,389 INFO: Bavaria gas power plant energy generation: 412579.15 MWh
2025-05-15 17:35:31,391 INFO: Bavaria gas power plant emissions: 716868708.96 kg CO2e
2025-05-15 17:35:31,391 INFO: Bavaria gas power plant emission intensity: 1737.53 kg CO2e/MWh


NRW Gas Power Plant peak load 40 MW


In [11]:
NRW_plant_capacity = PowerPlantAnalyzer.select_client_plant_value(powerco_df, 'Power Plant type', 'Gas (Peak load)', 'Power Capacity (MW)')
NRW_plant_energy = EnergyCalculator.calculate_energy_generation(NRW_plant_capacity, config.PEAK_CAPACITY_FACTOR)
NRW_plant_emissions = EmissionCalculator.calculate_emission(gas_plants_avg_kg_emission_per_mw, NRW_plant_capacity)
NRW_emission_intensity = EmissionCalculator.calculate_emission_intensity(NRW_plant_emissions, NRW_plant_energy)
logger.info("NRW gas power plant energy generation: %.2f MWh", NRW_plant_energy)
logger.info("NRW gas power plant emissions: %.2f kg CO2e", NRW_plant_emissions)
logger.info("NRW gas power plant emission intensity: %.2f kg CO2e/MWh", NRW_emission_intensity)

2025-05-15 17:35:31,408 INFO: NRW gas power plant energy generation: 45552.00 MWh
2025-05-15 17:35:31,416 INFO: NRW gas power plant emissions: 286747483.58 kg CO2e
2025-05-15 17:35:31,416 INFO: NRW gas power plant emission intensity: 6294.95 kg CO2e/MWh


North Sea Wind Offshore Park 50 MW 


In [12]:
wind_park_capacity = PowerPlantAnalyzer.select_client_plant_value(powerco_df, 'Power Plant type', 'Offshore Wind Park', 'Power Capacity (MW)')
wind_park_energy = EnergyCalculator.calculate_energy_generation(wind_park_capacity, avg_offshore_north_sea_capacity_factor)
wind_park_emissions = 0 #They do not produce emissions during operation, only those from the vessels used for service and maintenance. Source: https://us.orsted.com/renewable-energy-solutions/offshore-wind/seven-facts-about-offshore-wind/carbon-footprint
logger.info("Offshore wind park energy generation: %.2f MWh", wind_park_energy)
logger.info("Offshore wind park emissions: %.2f kg CO2e", wind_park_emissions)

2025-05-15 17:35:31,440 INFO: Offshore wind park energy generation: 192768.67 MWh
2025-05-15 17:35:31,448 INFO: Offshore wind park emissions: 0.00 kg CO2e


Bavaria Solar Park 10 MW 

In [13]:
solar_park_capacity = PowerPlantAnalyzer.select_client_plant_value(powerco_df, 'Power Plant type', 'Solar Park', 'Power Capacity (MW)')
solar_park_energy = EnergyCalculator.calculate_energy_generation(solar_park_capacity, avg_solar_capacity_factor)
solar_park_emissions = 0
logger.info("Solar park energy generation: %.2f MWh", solar_park_energy)
logger.info("Solar park emissions: %.2f kg CO2e", solar_park_emissions)

2025-05-15 17:35:31,473 INFO: Solar park energy generation: 10249.20 MWh
2025-05-15 17:35:31,473 INFO: Solar park emissions: 0.00 kg CO2e


PowerCo's overall physical emission intensity

In [14]:
powerco_generated_energy = bavaria_gas_energy  + NRW_plant_energy + wind_park_energy + solar_park_energy
powerco_emissions = bavaria_gas_emissions + NRW_plant_emissions + wind_park_emissions + solar_park_emissions
powerco_emission_intensity = EmissionCalculator.calculate_emission_intensity(powerco_emissions, powerco_generated_energy)
logger.info("PowerCo's physical emission intensity: %.2f kg CO2e/MWh", powerco_emission_intensity)

2025-05-15 17:35:31,497 INFO: PowerCo's physical emission intensity: 1517.99 kg CO2e/MWh


Calculate average emissions for the "Big 4" German electricity generators for benchmark

In [15]:
vattenfall_emission_intensity = EmissionCalculator.calculate_emission_intensity(config.VATTENFALL_EMISSIONS, config.VATTENFALL_GENERATION)
avg_emissions_big4 = (config.RWE_PHYSICAL_EMISSION_INTENSITY + config.ENBW_PHYSICAL_EMISSION_INTENSITY + config.EON_PHYSICAL_EMISSION_INTENSITY + vattenfall_emission_intensity) / 4
logger.info("Avg emissions of Big4: %.2f kg CO2e/MWh", avg_emissions_big4)

2025-05-15 17:35:31,527 INFO: Avg emissions of Big4: 287.74 kg CO2e/MWh
